# Label Free Quantification - LFQ (DDA)

In [ ]:
# Import required packages:

import msmu as mm
import pandas as pd

In [ ]:
# read TMT data (Sage format):
mdata = mm.read_sage("sage/output/dir/", label="label_free")

# MaxQuant format:
# mdata = mm.read_maxquant("path_to_maxquant_directory/", label="label_free")

# FragPipe format:
# mdata = mm.read_fragpipe("path_to_fragpipe_directory/", label="label_free")


In [ ]:
# (Optional) Add metadata for samples:

meta_df = pd.read_csv("path/to/metadata.csv")
meta_df = meta_df.set_index("tag")  # set the index to match sample names in mdata.obs

mdata.obs = mdata.obs.join(meta_df)
mdata.push_obs()  # push the updated obs back to the MuData object

In [ ]:
# Filter features based on q-value and (optional) contaminants:

mdata = mm.pp.add_filter(mdata, modality="feature", column="q_value", keep="lt", value=0.01)
# mdata = mm.pp.add_filter(mdata, modality="feature", column="proteins", keep="not contains", value="contam_")
mdata = mm.pp.apply_filter(mdata, modality="feature")

In [ ]:
# Summarise to peptide:

mdata = mm.pp.to_peptide(mdata)

In [ ]:
# Filter peptides based on q-value:

mdata = mm.pp.add_filter(mdata, modality="peptide", column="q_value", keep="lt", value=0.01)
mdata = mm.pp.apply_filter(mdata, modality="peptide")

In [ ]:
# Log2 transform the data:
mdata = mm.pp.log2_transform(mdata, modality="peptide")

In [ ]:
# Normalize the data using median centering normalization

mdata = mm.pp.normalise(mdata, modality="peptide", method="median")

In [ ]:
# Protein group inference with peptides and their matched proteins:

mdata = mm.pp.infer_protein(mdata)

In [ ]:
# Summarise to protein:
# Top 3 peptides wihin protein group can be used for protein group quantification aggregation (else, top_n=None)

mdata = mm.pp.to_protein(mdata, top_n=3, rank_method="total_intensity")

In [ ]:
# Filter protein groups based on q-value:

mdata = mm.pp.add_filter(mdata, modality="protein", column="q_value", keep="lt", value=0.01)
mdata = mm.pp.apply_filter(mdata, modality="protein")